In [1]:
# Reload logging for the Notebook

import importlib
import logging

importlib.reload(logging)

<module 'logging' from '/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/logging/__init__.py'>

In [2]:
# Configure root logger behaviour to output all log levels to stdout

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, format='%(levelname)s: %(message)s')

# Structured logging in Python

### Rob Van Gennip

linkedin.com/in/ravangen

# Rob Van Gennip

- Engineering Lead at Wave
- Over 3 years of using Python with Django
- Previously 4 years of C# and native Windows development
- Focus on distributed systems and tooling

# Topics

- Log Levels
- Logging Classes
- Structured Logging
- Tips 
- Questions

Save questions for the end

# Log Levels

- Debug
- Info
- Warning
- Error
- Critical

## Debug

Detailed internal state for diagnosing problems

In [3]:
import logging

logger = logging.getLogger()

def sum(x, y):
    result = x + y
    logger.debug(f'Caclulated sum of {x} and {y} as {result}')
    return result

sum(3, 4)

DEBUG: Caclulated sum of 3 and 4 as 7


7

Use instead of print function for developer output

Configuration can determine what is output  

Especially with a large amount of data or a high frequency, such as tracking an algorithm’s internal state

## Info

Confirmation things are working as expected

In [4]:
logger.info('User logged out')

INFO: User logged out


Understand what is happening in the application flow

A couple per operation, perhaps related to handling request or server state changed

Server started; event processed; user logged in

## Warning

Something unexpected happened or indicative of a problem for the near future

In [5]:
logger.warning('Unable to create short link')

Something important but not an error

Disk space low

## Error

Something is wrong, not able to perform an operation

In [6]:
logger.error('Error updating user', exc_info=False)

ERROR: Error updating user


Failure to read or create a database record

exc_info: indicates if exception information is added to the logging message

In [16]:
def divide_by_zero():
    x = 1/0

try:
    divide_by_zero()
except ZeroDivisionError:
    logger.exception('Cannot divide by zero')  # defaults exc_info=True

ERROR: Cannot divide by zero
Traceback (most recent call last):
  File "<ipython-input-16-d1f79933074c>", line 5, in <module>
    divide_by_zero()
  File "<ipython-input-16-d1f79933074c>", line 2, in divide_by_zero
    x = 1/0
ZeroDivisionError: division by zero


`exception` should only be called from an exception handler

exc_info accepts:
- boolean
- an exception instance
- an exception tuple in the format returned by sys.exc_info()

## Critical

Something really bad happened, may be unable to continue running

In [8]:
logger.critical('An unhandled exception was thrown by the application')

CRITICAL: An unhandled exception was thrown by the application


Out of memory

Disk is full

# Logging Classes

- Loggers
- Handlers
- Filters
- Formatters

## Logger

- The interface that application code directly uses
- All loggers are descendants of the root logger
- Named typically as a dot-separated hierarchy like `'a'`, `'a.b'` or `'a.b.c.d'`
- Retrieve an instance with `logging.getLogger(name)`
- Retrieving the same named logger returns a reference to the same instance

Each logger passes log messages on to its parent unless `propagate` is `False`

Logger never instantiated directly, but always through `getLogger`

In [9]:
root_logger = logging.getLogger()  # no name provided

root_logger.info('Hello PyCon Canada!')

INFO: Hello PyCon Canada!


In [10]:
foo_bar_logger = logging.getLogger('foo.bar')

foo_bar_logger.info('Hello PyCon Canada!')

INFO: Hello PyCon Canada!


In [11]:
# Use __name__ to get the fully-qualified name of the module
# Example: api/user.py → 'api.user'
# __name__ is '__main__' in an interactive prompt, script, or standard input

auto_named_logger = logging.getLogger(__name__)  # recommended

auto_named_logger.info('Hello PyCon Canada!')

INFO: Hello PyCon Canada!


Using `__name__` ensures no unexpected name collisions

## Handler

- Sends the log records to a destination
- `StreamHandler`: output to streams such as `stdout`, `stderr`
- `NullHandler`: no output
- `FileHandler`: output to a disk file
- `RotatingFileHandler`: rotates at max file size
- `TimedRotatingFileHandler`: interval based rotation
- `SmtpHandler`: send an email per record

`RotatingFileHandler`: old log files by append the extensions .1, .2 to the file name

## Filter

- Fine grained mechanism for determining which log records to output
- Returns a true value if the record is to be processed

In [12]:
TESTING = True


class NotInTestingFilter(logging.Filter):

    def filter(self, record):
        return not TESTING

## Formatter

- Specify how content of log records are transformed for output
- Format should at least include current time, level, name, and message

#### JSON Formatter

- Output each record as JSON object
    - Includes each format string argument as key/value
    - Merge `extra` dict into output object
- Conveniently machine parsable
- Stop writing custom parsers for records

In [13]:
! pip3 install python-json-logger

In [14]:
server_logger = logging.getLogger('server')
server_logger.propagate = False  # do not propagate messages up logger hierarchy

log_format = '%(asctime)s - %(levelname)s - %(request_id)s - %(message)s'
formatter = logging.Formatter(log_format)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(formatter)
server_logger.addHandler(handler)

data = {
    'request_id': '90fca134-4468-41dc-bb26-4d087d61857b',
    'duration': 42,  # not output, not in log_format
}

server_logger.info('User logout', extra=data)

# NOTE: request_id must always be provided to 'server' logger
# If not provided, a KeyError will be raised by the format string

2017-11-18 13:27:54,997 - INFO - 90fca134-4468-41dc-bb26-4d087d61857b - User logout


In [15]:
from pythonjsonlogger import jsonlogger

api_logger = logging.getLogger('api')
api_logger.propagate = False  # do not propagate messages up logger hierarchy

log_format = '%(asctime)s %(levelname)s %(message)s'  # only standard attributes
formatter = jsonlogger.JsonFormatter(log_format)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(formatter)
api_logger.addHandler(handler)

data = {
    # output despite not in log_format
    'request_id': '90fca134-4468-41dc-bb26-4d087d61857b',
    'duration': 42,
}

api_logger.info('User logout', extra=data)

{"asctime": "2017-11-18 13:27:55,028", "levelname": "INFO", "message": "User logout", "request_id": "90fca134-4468-41dc-bb26-4d087d61857b", "duration": 42}


It doesn’t really take any longer to log custom properties as you write your logging

The extra properties can provide more details that make it easier to troubleshoot application problems


# https://docs.python.org/3/library/logging.html#logrecord-attributes
# %(asctime)s : Human-readable time when the LogRecord was created
# %(levelname)s : Text logging level for the message
# %(name)s : Name of the logger used to log the call
# %(message)s : The logged message

# Structured Logging

- Records key/value pairs
- Output is readable by humans and computers
- Build context around operations
- Eliminate any guesswork, let tools format and adapt to the data
- Not specific to JSON, logfmt alternative:
```
level=info msg="Stopping all fetchers"
    tag=stopping_fetchers id=ConsumerFetcherManager-1382721708341
    module=kafka.consumer.ConsumerFetcherManager
```

A general problem with log files is they are unstructured text data, making it hard to query them

## Use Cases

- Debugging and searching across every app and server
    - Record unique identifiers like customer or transaction id
- Monitoring the infrastructure’s health and security
    - Looking at frequency and volume, trying to detect anomalies
- Generate meaningful reports and fuel data products
    - E.g. server access logs to do some summarization and aggregation

When using a log management system that supports searching by custom fields, then search becomes easy

Quickly narrow down problems for a specific client

Unique IDs can point to the exact transaction, otherwise you might only have a time range to use

Carry these IDs through multiple touch points to track transactions through the system and follow them across machines, networks, and services

## Infrastructure

### Open Source
- **E**lasticsearch: storage and search engine
- **L**ogstash: filtering and forwarding
- **K**ibana: analysis and visualization

### Commercial
- Sumo Logic
- Splunk

TODO: diagram

- Forwarder
- Collector/Filter
- Storage

Add columns with "Table icon"
- `apache2.access.url`
- `apache2.access.response_code`
- `apache2.access.user_agent.name`
- `apache2.access.user_agent.os`

Find matching results with "Magnifying icon"



# Tips

- Be concise and descriptive
- Include a human readable message
- Include extra fields (especially for exceptions)
- Do not to log passwords and any personal information
- When logging in a library, the system should dictate the logging configuration

A logging statement should contain both data and description

Trying to figure out why an exception happened is infinitely easier if you know more details about who the user was and the input parameters

It is important to have complete logs and to trust them

- Make use of `extra` parameter
- `stack_info`

# Questions